<a href="https://colab.research.google.com/github/jdeept/Financial-News/blob/main/Jerom_Powell_Speech_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers yfinance
!pip install nltk

In [2]:
speech_text = """
Good afternoon. Today the Federal Reserve will continue its commitment to bringing down inflation...
"""

FIBERNET

In [3]:
from transformers import pipeline

nlp = pipeline("sentiment-analysis", model="ProsusAI/finbert")
results = nlp(speech_text)
print(results)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': 'positive', 'score': 0.7853304743766785}]


In [9]:
import yfinance as yf
from datetime import datetime, timedelta

def analyze_speech(date_str, coin="BTC-USD", window=5):
    """
    Analyze BTC (or any coin) price change around a speech date.

    date_str : str   -> 'YYYY-MM-DD' format (speech date)
    coin     : str   -> default 'BTC-USD'
    window   : int   -> days before & after speech
    """
    # Convert date string to datetime
    speech_date = datetime.strptime(date_str, "%Y-%m-%d")

    # Expand range by 'window' days before and after
    start = (speech_date - timedelta(days=window)).strftime("%Y-%m-%d")
    end   = (speech_date + timedelta(days=window)).strftime("%Y-%m-%d")

    # Download data
    data = yf.download(coin, start=start, end=end, interval="1d")

    if data.empty:
        print("⚠️ No data found for this range.")
        return None

    # Take price before and after
    price_before = data['Close'].iloc[0]
    price_after  = data['Close'].iloc[-1]

    change = ((price_after - price_before) / price_before) * 100

    print(f"📅 Speech Date: {date_str}")
    print(f"💰 Coin: {coin}")
    print(f"📊 From {data.index[0].date()} to {data.index[-1].date()}")
    print(f"BTC % Change: {round(change, 2)}%")

    return change

# ✅ Example usage:
analyze_speech("2023-06-01")  # Powell speech date


/tmp/ipython-input-1366460593.py:20: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(coin, start=start, end=end, interval="1d")
[*********************100%***********************]  1 of 1 completed

📅 Speech Date: 2023-06-01
💰 Coin: BTC-USD
📊 From 2023-05-27 to 2023-06-05
BTC % Change: Ticker
BTC-USD   -4.12
dtype: float64%


,0
Ticker,
BTC-USD,-4.124763


Sentimental Analysis

In [10]:
import requests

url = "https://www.federalreserve.gov/newsevents/speech/files/powell20250822.pdf"
response = requests.get(url)
with open("powell_jh_20250822.pdf", "wb") as f:
    f.write(response.content)


In [11]:
!pip install PyPDF2
import PyPDF2

with open("powell_jh_20250822.pdf", "rb") as f:
    reader = PyPDF2.PdfReader(f)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"

print(text[:500])  # preview the first 500 characters


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.9 MB/s eta 0:00:00
For release on delivery  
10:00 a.m. EDT ( 8:00 a.m. MDT ) 
August 22, 2025 
 
       
Monetary Policy and the Fed’s  Framework Review  
   
Remarks by  
 
Jerome H. Powell  
 
Chair  
 
Board of Governors of the Federal Reserve System  
 
at 
 
“Labor Markets in Transition:  Demographics, Productivity, and Macroeconomic Policy,”  
an economic symposium sponsored by the Federal Reserve Bank of Kansas City  
 
Jackson Hole, Wyoming  
       
August 22, 2025
 
Over the course of this year, the U.S


In [13]:
from transformers import pipeline

# Load FinBERT
sentiment = pipeline("sentiment-analysis", model="ProsusAI/finbert")

# Break text into smaller parts (e.g., ~300 words per chunk)
def chunk_text(text, max_words=300):
    words = text.split()
    return [" ".join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

chunks = chunk_text(text)

# Run sentiment analysis on each chunk
results = sentiment(chunks)

# Summarize results
pos, neg, neu = 0, 0, 0
for r in results:
    if r['label'] == 'positive':
        pos += 1
    elif r['label'] == 'negative':
        neg += 1
    else:
        neu += 1

print("📊 FinBERT Sentiment Summary:")
print(f"Positive: {pos}, Negative: {neg}, Neutral: {neu}")


Device set to use cuda:0


📊 FinBERT Sentiment Summary:
Positive: 1, Negative: 4, Neutral: 12


Hawkish - Negative News , Dovish - Positive news